# Base de Datos de PROCEDIMIENTOS
### Leer base de datos

leer la base de datos "data/Base_informada" 


In [10]:
import re
import pandas as pd 
from openpyxl import load_workbook
import numpy as np

In [11]:
LUGARES_CATALOGADOS= {
                    "SECTOR RESTRINGIDO":	"SECTOR DE SEGURIDAD RESTRINGIDA AEROPORTUARIA",
                    "SECTOR PUBLICO"	:"INSTALACIONES DE ACCESO AL PUBLICO GENERAL",
                    "PERIMETRO DE JURISDICCION":	"PERIMETRO AEROPORTUARIO",
                    "FUERA DE JURISDICCION":	"DESPLIEGUE",
                    "TRANSPORTE PUBLICO": "TRANSPORTE PUBLICO",
                    "S/D": "S/D"
}

UNIDADES_MUNICIPIOS = {
    "EZE": "JOSÉ M. EZEIZA",
    "AER": "COMUNA 14",
    "SFO": "SAN FERNANDO",
    "BHI": "BAHÍA BLANCA",
    "MDP": "GENERAL PUEYRREDÓN",
    "MDZ": "LAS HERAS",
    "COR": "CAPITAL",
    "JUA": "9 DE JULIO",
    "LUI": "JUAN MARTÍN DE PUEYRREDÓN",
    "MLG": "MALARGÜE",
    "RAF": "SAN RAFAEL",
    "RCU": "RÍO CUARTO",
    "SRO": "CAPITAL",
    "JUJ": "EL CARMEN",
    "SAL": "LA CAPITAL",
    "CAT": "VALLE VIEJO",
    "LAR": "CAPITAL",
    "SGO": "CAPITAL",
    "TRH": "RÍO HONDO",
    "TUC": "CRUZ ALTA",
    "IGU": "IGUAZÚ",
    "FSA": "FORMOSA",
    "ROS": "ROSARIO",
    "RCQ": "GENERAL OBLIGADO",
    "POS": "CAPITAL",
    "CRR": "CAPITAL",
    "LIB": "PASO DE LOS LIBRES",
    "PAR": "PARANÁ",
    "RES": "SAN FERNANDO",
    "SVO": "LA CAPITAL",
    "BAR": "BARILOCHE",
    "TRE": "RAWSON",
    "NEU": "CONFLUENCIA",
    "CAL": "LAGO ARGENTINO",
    "CHP": "LÁCAR",
    "CRV": "ESCALANTE",
    "PMY": "BIEDMA",
    "ESQ": "FUTALEUFÚ",
    "GAL": "GÜER AIKE",
    "GDE": "RÍO GRANDE",
    "USU": "USHUAIA",
    "VIE": "ADOLFO ALSINA",
    "RG4": "-",
}

PROVINCIAS = {
    "C.A. BUENOS AIRES": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "BUENOS AIRES": "BUENOS AIRES",
    "SANTA CRUZ": "SANTA CRUZ",
    "SANTA FE": "SANTA FE",
    "MENDOZA": "MENDOZA",
    "TUCUMÁN": "TUCUMAN",
    "TIERRA DEL FUEGO": "TIERRA DEL FUEGO ANTARTIDA E ISLAS DEL ATLANTICO SUR",
    "SALTA": "SALTA",
    "NEUQUÉN": "NEUQUEN",
    "CÓRDOBA": "CORDOBA",
    "LA PAMPA": "LA PAMPA",
    "RÍO NEGRO": "RIO NEGRO",
    "CATAMARCA": "CATAMARCA",
    "MISIONES": "MISIONES",
    "SAN JUAN": "SAN JUAN",
    "CORRIENTES": "CORRIENTES",
    "CHUBUT": "CHUBUT",
    "FORMOSA": "FORMOSA",
    "JUJUY": "JUJUY",
    "SANTIAGO DEL ESTERO": "SANTIAGO DEL ESTERO",
    "LA RIOJA": "LA RIOJA",
    "CHACO": "CHACO",
    "ENTRE RÍOS": "ENTRE RIOS",
    "SAN LUIS": "SAN LUIS"
}



PERSONAL = {    
    "DENUNCIA": 4,
    "CONTROL PREVENTIVO": 6,
    "PATIO VALIJAS": 6,
    "ALLANAMIENTO" :8,
}

SCANNER = {
    "DENUNCIA": 4,
    "CONTROL PREVENTIVO": 6,
    "PATIO VALIJAS": 6,
    "ALLANAMIENTO" :8,
}

# Listas de prefijos conocidos
PREFIJOS = ["DOL", "OSR", "OSC", "OSL", "AP","FCB","RL","AJ","PS", 
            "CPE", "CSJ","CCC","CPF", "CPN","FBB", "FCR", "FCB",
            "FCT","FGR","FLP", "FMP","FMZ","FPO","FPA","FRE","FCA","FRO",
            "FSM","FTU", "CUIJ", "COIRON"]

# Unir las ubicaciones conocidas del diccionario `UNIDADES_MUNICIPIOS` con las anteriores
UBICACIONES = [
    "EZE", "AER", "SFO", "BHI", "MDP", "MDZ", "COR", "JUA", "LUI", "MLG", "RAF", 
    "RCU", "SRO", "JUJ", "SAL", "CAT", "LAR", "SGO", "TRH", "TUC", "IGU", "FSA",
    "ROS", "RCQ", "POS", "CRR", "LIB", "PAR", "RES", "SVO", "BAR", "TRE", "NEU",
    "CAL", "CHP", "CRV", "PMY", "ESQ", "GAL", "GDE", "USU", "VIE", "RG4",
    "UR1", "UR2", "UR3", "UR4", "UR5", "RG1", "RG2", "RG3", "RG5", "TO1", "TO2"
]

* Funciones de procedimientos 

In [12]:

def generar_uid_sigpol(row):
        tipo = str(row['TIPO_CAUSA_INTERNA'])
        uosp = str(row['UOSP'])
        numero_parte = str(row['NUMERO_PARTE'])
        anio = str(row['ANIO_PARTE'])
        return tipo +"-"+ uosp + "-"+ numero_parte+"-"+anio


def procesar_descripcion(row):
    tipo = row['TIPO_PROCEDIMIENTO']
    if tipo == "DENUNCIA":
        return "DENUNCIA POLICIAL"
    elif tipo == "CONTROL PREVENTIVO":
        return f"CONTROL PREVENTIVO - {procesar_lugar(row)}"
    elif tipo == "ORDEN DE ALLANAMIENTO":
        return "ORDEN DE ALLANAMIENTO"
    elif tipo == "ORDEN DE ALLANAMIENTO / DETENCIÓN":
        return "ORDEN DE ALLANAMIENTO"
    else:
        return "OTRO MANDATO JUDICIAL"

def procesar_tipo(row):
    tipo = row['TIPO_PROCEDIMIENTO']
    if pd.isna(tipo):
        return ""
    elif tipo == "DENUNCIA"  or tipo == "CONTROL PREVENTIVO" :
        return "ORDEN POLICIAL"
    else:
        return "ORDEN JUDICIAL"


def procesar_provincia(row):
    provincia = row['PROVINCIA']
    jurisdiccion = row['JURISDICCION']
    if pd.isna(provincia):
        return PROVINCIAS.get(jurisdiccion, provincia)
    else:
        return PROVINCIAS.get(provincia, provincia)
        
        
def procesar_municipio(row):
    unidad = row['UOSP']
    if pd.isna(unidad):
        return ""
    return UNIDADES_MUNICIPIOS.get(unidad, unidad)

def procesar_lugar(row):
    lugar = row['LUGAR_CATALOGADO_NIVEL_1']
    if pd.isna(lugar):
        return "-"
    return  LUGARES_CATALOGADOS[lugar]

def procesar_direccion(row):
    # Asignar valores y manejar el caso en que sean None o NaN
    lugar = row['LUGAR_CATALOGADO_NIVEL_1'] if row['LUGAR_CATALOGADO_NIVEL_1'] is not None else ""
    calle = str(row['CALLE']) if row['CALLE'] is not None else ""
    numero = str(row['NUMERO']) if row['NUMERO'] is not None else ""
    ciudad = str(row['CIUDAD']) if row['CIUDAD'] is not None else ""
    partido = str(row['PARTIDO']) if row['PARTIDO'] is not None else ""

    # Lógica para construir la dirección basada en las condiciones
    if lugar == "FUERA DE JURISDICCION" and ciudad == "ROSARIO":
        return ""
    elif lugar == "FUERA DE JURISDICCION":
        return calle + " " + numero + ", " + ciudad + " - " + partido
    else:
        return "-"

def controlar_estado (row):
    ursa = row['URSA']
    unidad = row['UOSP']
    estado = row['ESTADO_PARTE']
    if pd.isna(unidad)  and ursa == 'RG4' and estado == 'NO DISPONIBLE ESTADISTICA':
        return  "DISPONIBLE ESTADISTICA"
    else:
        return estado


def leer_excel_a_df(worksheet):
    titulos = [worksheet.cell(row=3, column=col).value for col in range(2, worksheet.max_column + 1)]

    data = []
    for row in worksheet.iter_rows(min_row=4, min_col=2, max_col=worksheet.max_column, values_only=True):
        data.append(row)

    df = pd.DataFrame(data, columns=titulos)
    return df


def procesar_causa_judicial(row):
    # Obtener los valores de las columnas, asegurándose de que no sean `None` o `NaN`
    causa = str(row.get('CAUSAJUDICIALNUMERO', '')).strip()
    tipo = str(row.get('TIPO_CAUSA_INTERNA', '')).strip()
    causa_int = str(row.get('CAUSA_INTERNA_NUMERO', '')).strip()


    # Validar si el campo 'causa' tiene un valor no deseado
    if causa in ["", "S/D", "A/S", "N/C", "None","nan", "--", "SN°","S/N","-", "---"]:
        # Retornar el tipo y el número de causa interna, asegurando que no haya guiones redundantes
        resultado = f"{tipo}-{causa_int}".replace("--", "-").strip("-")
        return resultado

    # Definir los prefijos que deben ser eliminados
    prefijos = ["NRO", "N°", "EXPTE", "EXPEDIENTE", "EXPT", "N ", '.', ':', '"', '_', '´', "`","°",""]
    
    # Crear una expresión regular para eliminar los prefijos
    
    causa = causa.strip()
    for prefijo in prefijos:
        causa = causa.replace(prefijo, "").strip()

    # Normalizar la causa, añadiendo guion entre letras y números si es necesario
    causa = re.sub(r'([A-Za-z]+)\s*(\d+)', r'\1-\2', causa)

    # Eliminar guiones dobles o triples si existen
    causa = re.sub(r'-{2,}', '-', causa)

    return causa


In [13]:
# Lista de años
años = [ 2023, 2024]

# Lista de arrays
arrays = []

# Leer cada archivo Excel y agregarlo a la lista de arrays
for año in años:
    archivo = f"data/BASE_{año}.xls"
    array = pd.read_excel(archivo).values
    arrays.append(array)

# Concatenar los arrays para crear el DataFrame final
compilado = np.concatenate(arrays, axis=0)

# Convertir el array a un DataFrame
compilado = pd.DataFrame(compilado)

# Establecer los nombres de las columnas
compilado.columns = pd.read_excel(f"data/BASE_{años[0]}.xls").columns


compilado["UID"] = compilado["NUMERO_PARTE"].astype(str) + "/" + compilado["ANIO_PARTE"].astype(str)

print(f"Antes de duplicados: {compilado['UID'].count()}")

compilado.drop_duplicates(subset='UID', keep='first', inplace=True)

print(f"Después de duplicados: {compilado['UID'].count()}")

# Reemplazar valores específicos en la columna "tipo de causa"
compilado['TIPO_CAUSA_INTERNA'] = compilado['TIPO_CAUSA_INTERNA'].replace({
    'RESTRICCIÓN A LA LIBERTAD': 'RL',
    'ACTUACIÓN JUDICIAL': 'AJ',
    'ACTUACIONES JUDICIALES': 'AJ',
})

compilado['ESTADO_PARTE'] = compilado.apply(controlar_estado ,axis=1).copy()
compilado['UOSP'] = compilado['UOSP'].fillna(compilado['URSA'])
compilado['GEOREFERENCIA_X'] = compilado['GEOREFERENCIA_X'].fillna('-')
compilado['GEOREFERENCIA_Y'] = compilado['GEOREFERENCIA_Y'].fillna('-')


print(f"Antes de eliminación de 'NO DISPONIBLE ESTADISTICA': {compilado['UID'].count()}")

compilado = compilado[compilado['ESTADO_PARTE'] != "NO DISPONIBLE ESTADISTICA"]

print(f"Después de eliminación de 'NO DISPONIBLE ESTADISTICA': {compilado['UID'].count()}")


Antes de duplicados: 9230
Después de duplicados: 9189
Antes de eliminación de 'NO DISPONIBLE ESTADISTICA': 9189
Después de eliminación de 'NO DISPONIBLE ESTADISTICA': 8738


* Filtro cargo lo ya informado de operaciones y lo que se informo lo concateno  a lo de operaciones de 2024

In [14]:
Base_procedimientos = pd.DataFrame()
Base_procedimientos['UID'] = compilado.apply(generar_uid_sigpol, axis=1)
Base_procedimientos['ID_OPERATIVO'] = compilado.apply(procesar_causa_judicial, axis=1)
Base_procedimientos['FUERZA_INTERVINIENTE'] = "PSA"
Base_procedimientos['ID_PROCEDIMIENTO'] = Base_procedimientos['UID']
Base_procedimientos['CAUSAJUDICIALNUMERO'] = compilado['CAUSAJUDICIALNUMERO'].copy()
Base_procedimientos['UNIDAD_INTERVINIENTE'] = compilado['UOSP']
Base_procedimientos['DESCRIPCIÓN'] = compilado.apply(procesar_descripcion, axis=1)
Base_procedimientos['TIPO_INTERVENCION'] = compilado.apply(procesar_tipo, axis=1)
Base_procedimientos['PROVINCIA'] = compilado.apply(procesar_provincia, axis=1)
Base_procedimientos['DEPARTAMENTO O PARTIDO'] = compilado.apply(procesar_municipio, axis=1)
Base_procedimientos['LOCALIDAD'] = "-"
Base_procedimientos['DIRECCION'] = compilado.apply(procesar_direccion, axis=1)
Base_procedimientos['LATITUD'] = compilado['GEOREFERENCIA_Y']
Base_procedimientos['LONGITUD'] = compilado['GEOREFERENCIA_X']
Base_procedimientos['FECHA'] = pd.to_datetime(compilado['DENUNCIAFECHA'], errors='coerce').dt.strftime('%d/%m/%Y')
Base_procedimientos['HORA'] = pd.to_datetime(compilado['DENUNCIAFECHA'], errors='coerce').dt.strftime('%H:%M')
Base_procedimientos['ZONA_SEGURIDAD_FRONTERAS'] = "-"
Base_procedimientos['PASO_FRONTERIZO'] = "-"
Base_procedimientos['OTRAS AGENCIAS INTERVINIENTES'] = "-"
Base_procedimientos['Observaciones - Detalles'] = "-"

Base_procedimientos = Base_procedimientos[[ "UID",'FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO',
                                     'UNIDAD_INTERVINIENTE', 'DESCRIPCIÓN', 'TIPO_INTERVENCION',
                                     'PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION',
                                     'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD',
                                     'FECHA', 'HORA', 'OTRAS AGENCIAS INTERVINIENTES', 'Observaciones - Detalles']]


Base_procedimientos.to_excel("data/Base_procedimientos_2023_2024.xlsx",index= False)

# Convertir la columna 'FECHA' a formato datetime para poder aplicar los filtros
# Base_procedimientos['FECHA'] = pd.to_datetime(Base_procedimientos['FECHA'], format='%d/%m/%Y', errors='coerce')

# Filtrar los registros desde septiembre hacia atrás (incluido)
# filtro_hasta_septiembre = Base_procedimientos['FECHA'] <= '2024-09-30'
# base_procedimientos_hasta_septiembre = Base_procedimientos[filtro_hasta_septiembre]

# Filtrar los registros a partir de octubre
# filtro_octubre_en_adelante = Base_procedimientos['FECHA'] >= '2024-10-01'
# base_procedimientos_octubre_en_adelante = Base_procedimientos[filtro_octubre_en_adelante]

# Guardar ambos DataFrames en archivos separados para su revisión
# base_procedimientos_hasta_septiembre.to_excel("Base_procedimientos_hasta_septiembre_2024.xlsx", index=False)
# base_procedimientos_octubre_en_adelante.to_excel("Base_procedimientos_octubre_en_adelante_2024.xlsx", index=False)







In [48]:
def filtrar_procedimientos (ruta_archivo):
    cantidad_partes_inical = 0
    cantidad_partes = 0
    cantidad_partes_duplicados = 0
    cantidad_partes_no_disponible = 0
    
    df = pd.read_excel(ruta_archivo)
    df["UID"] = df["NUMERO_PARTE"].astype(str) + "/" + df["ANIO_PARTE"].astype(str)
    
    cantidad_partes_inical = df['UID'].count()
    cantidad_partes = df['UID'].count()

    df.drop_duplicates(subset='UID', keep='first', inplace=True)
    
    
    
    cantidad_partes_duplicados = cantidad_partes - df['UID'].count()
    cantidad_partes = df['UID'].count()

    # Reemplazar valores específicos en la columna "tipo de causa"
    df['TIPO_CAUSA_INTERNA'] = df['TIPO_CAUSA_INTERNA'].replace({
        'RESTRICCIÓN A LA LIBERTAD': 'RL',
        'ACTUACIÓN JUDICIAL': 'AJ',
        'ACTUACIONES JUDICIALES': 'AJ',
    })

    df['ESTADO_PARTE'] = df.apply(controlar_estado ,axis=1).copy()
    df['UOSP'] = df['UOSP'].fillna(df['URSA'])
    df['GEOREFERENCIA_X'] = df['GEOREFERENCIA_X'].fillna('-')
    df['GEOREFERENCIA_Y'] = df['GEOREFERENCIA_Y'].fillna('-')
    
    df = df[df['ESTADO_PARTE'] != "NO DISPONIBLE ESTADISTICA"]

    
    df["UID"] = df.apply(generar_uid_sigpol,axis=1)
    
    cantidad_partes_no_disponible = cantidad_partes - df['UID'].count()
    cantidad_partes = df['UID'].count()
    
    print(f"Estadistica de Partes\n")
    print(f"Cantidad de Partes inicial: {cantidad_partes_inical}" )
    print(f"Cantidad Duplicado: {cantidad_partes_duplicados}" )
    print(f"Cantidad No diponible: {cantidad_partes_no_disponible}" )
    print(f"Cantidad de Partes final: {cantidad_partes}" )
    
    return df
    
partes_ingresados = filtrar_procedimientos("bajadas/bajada_general.xls")







Estadistica de Partes

Cantidad de Partes inicial: 491
Cantidad Duplicado: 3
Cantidad No diponible: 23
Cantidad de Partes final: 465


In [ ]:
df_sin_coincidencias = partes_ingresados[~partes_ingresados['UID'].isin(Base_procedimientos['UID'])]

UID PS-EZE-4150-2024  encontrado en Base_procedimientos
UID PS-SAL-4154-2024  encontrado en Base_procedimientos
UID C-EZE-4155-2024  encontrado en Base_procedimientos
UID AP-RG4-4156-2024  encontrado en Base_procedimientos
UID PS-BAR-4157-2024  encontrado en Base_procedimientos
UID PS-COR-4158-2024  encontrado en Base_procedimientos
UID PS-NEU-4160-2024  encontrado en Base_procedimientos
UID PS-COR-4161-2024  encontrado en Base_procedimientos
UID AP-MDP-4162-2024  encontrado en Base_procedimientos
UID PS-NEU-4163-2024  encontrado en Base_procedimientos
UID PS-COR-4164-2024  encontrado en Base_procedimientos
UID PS-IGU-4165-2024  encontrado en Base_procedimientos
UID PS-RAF-4166-2024  encontrado en Base_procedimientos
UID AP-AER-4167-2024  encontrado en Base_procedimientos
UID AP-RG4-4168-2024  encontrado en Base_procedimientos
UID PS-AER-4169-2024  encontrado en Base_procedimientos
UID PS-NEU-4170-2024  encontrado en Base_procedimientos
UID PS-SAL-4171-2024  encontrado en Base_procedim